In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ranking.pagerank import *
from ranking.hits import *
from ranking.metrics import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
df = pd.read_csv("res\\movie\\rating.csv")

In [4]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [5]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [7]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [8]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [9]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


Правило обновления в функции __get_pagerank
$$ \vec{x} \leftarrow a P \vec{x} + b \begin{bmatrix} p^{(u)} \vec{x} \\ p^{(u)} \vec{x} \\ \vdots \\ p^{(u)} \vec{x} \end{bmatrix} + c \begin{bmatrix} \frac{1}{n} \cdots \frac{1}{n} \end{bmatrix}^{T}$$

In [10]:
pg_t = PagerankFactory().read_txt('./res/google_graph/web-Google.txt', 1)
pred = pg_t.get_pagerank()
print(pred.sort_values(by='pagerank', ascending=False).head())

              0       1
0             0   11342
1             0  824020
2             0  867923
3             0  891835
4         11342       0
...         ...     ...
5105034  916425  547323
5105035  916425  604833
5105036  916425  632916
5105037  916425  637936
5105038  916425  837379

[5105039 rows x 2 columns]


C:\Users\odo20\AppData\Roaming\Python\Python312\site-packages\scipy\sparse\_base.py:713: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)
100%|██████████| 100/100 [00:24<00:00,  4.02it/s]


      target  pagerank
2529  163075  0.000036
3144  537039  0.000032
114   597621  0.000031
3146  605856  0.000028
8010  819223  0.000026


In [11]:
pg = PagerankFactory().read_pd(data, 'userId', 'movieId', 'rating', 1)

In [12]:
prediction_movie_for_user1 = pg.get_pagerank()

100%|██████████| 100/100 [01:45<00:00,  1.06s/it]


In [13]:
print(prediction_movie_for_user1.sort_values(by='pagerank', ascending=False).head())


     target  pagerank
153     318  0.000767
592     296  0.000750
305     356  0.000708
49      593  0.000704
92      260  0.000579


In [14]:
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~(prediction_movie_for_user1['target'].isin(train1['movieId']))]
print(len(prediction_movie_for_user1_without_famous))
prediction_movie_for_user1_without_famous.head()

26657


,target,pagerank
87,3,0.000117
88,62,0.000206
89,70,0.000093
90,110,0.000571
91,242,0.000009


In [15]:
# Топ фильмов по мнению pagerank
pred = prediction_movie_for_user1_without_famous.sort_values(by='pagerank', ascending=False).head(len(test1))
print(len(pred))
pred.head()

88


,target,pagerank
153,318,0.000767
592,296,0.000750
305,356,0.000708
92,260,0.000579
95,480,0.000575


In [16]:
def accuracy_without_order(true_labels, predicted_labels):
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set)
    
    return accuracy

In [17]:
print(len(test1['movieId']))
test1['movieId'].head()

88


156    7045
145    6242
101    3081
127    4911
141    5898
Name: movieId, dtype: int64

In [18]:
print(len(pred['target']))
pred['target'].head()

88


153    318
592    296
305    356
92     260
95     480
Name: target, dtype: int64

In [19]:
print(accuracy_without_order(test1['movieId'], pred['target']))

0.25


In [20]:
movie_df = pd.read_csv('res\\movie\\movie.csv')

In [21]:
prediction_moviename = pd.merge(movie_df, pred, left_on='movieId', right_on='target', how='inner')
prediction_moviename.sort_values(by='pagerank', ascending=False).head()

,movieId,title,genres,target,pagerank
21,318,"Shawshank Redemption, The (1994)",Crime|Drama,318,0.000767
19,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,296,0.000750
26,356,Forrest Gump (1994),Comedy|Drama|Romance|War,356,0.000708
16,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,260,0.000579
33,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,480,0.000575


In [22]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
print(len(test_moviename))
test_moviename.sort_values(by='rating', ascending=False).head()

88


,movieId,title,genres,userId,rating,timestamp
73,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,1,5.0,2005-04-02 23:30:19
87,8636,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,1,4.5,2005-04-02 23:44:53
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
54,3499,Misery (1990),Drama|Horror|Thriller,1,4.0,2005-04-02 23:35:18


In [23]:
# Фильмы которые были угаданны
test_moviename[test_moviename['movieId'].isin(prediction_moviename['movieId'])]

,movieId,title,genres,userId,rating,timestamp
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,1,3.5,2005-04-02 23:33:39
2,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,3.5,2005-04-02 23:29:40
4,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1,4.0,2005-04-02 23:33:46
5,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,1,4.0,2005-04-02 23:31:43
6,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1,4.0,2005-04-02 23:32:47
7,318,"Shawshank Redemption, The (1994)",Crime|Drama,1,4.0,2005-04-02 23:33:18
8,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1,4.0,2005-04-02 23:30:03
9,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,1,3.5,2005-04-02 23:45:57
12,1036,Die Hard (1988),Action|Crime|Thriller,1,4.0,2005-04-02 23:44:40
15,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller,1,3.5,2005-04-02 23:31:09


In [24]:
for_score = prediction_movie_for_user1.merge(test1, how='inner', left_on='target', right_on='movieId')
o = O(test1['rating'].mean())
t = T(prediction_movie_for_user1['pagerank'].mean())
i = I(t, o)
pairroid = Pairord(i)
print(pairroid(for_score['pagerank'], for_score['rating']))

0.6701962809917356


In [25]:
prec = Prec(o, t)
print(prec(for_score['pagerank'], for_score['rating']))

0.525


In [26]:
rec = Rec(o, t)
print(rec(for_score['pagerank'], for_score['rating']))

0.9333333333333333


In [27]:
f1 = F1(prec, rec)
print(f1(for_score['pagerank'], for_score['rating']))

0.672


In [28]:

pred = np.array(for_score['pagerank'])
true = np.array(for_score['rating'])
pred_map = np.vectorize(lambda x: t(x))
true_map = np.vectorize(lambda x: o(x))

pred_trasformed = pred_map(pred)
true_trasformed = true_map(true)

In [29]:
# Вычисляем метрики
accuracy = accuracy_score(true_trasformed, pred_trasformed)
precision = precision_score(true_trasformed, pred_trasformed)
recall = recall_score(true_trasformed, pred_trasformed)
f1 = f1_score(true_trasformed, pred_trasformed)

# Выводим результаты
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5340909090909091
Precision: 0.525
Recall: 0.9333333333333333
F1 Score: 0.672
